## Data Base Cleaning 

In this notebook we document the process of detecting and correcting (or removing) corrupt or inaccurate records from our PlacetoPay database. The process include identifying incomplete, incorrect, inaccurate or irrelevant parts of the data and then replacing, modifying, or deleting the dirty or coarse data. 

The shape of the original database was 47 columns and was 12 325 791 rows. After some deliberation we decided that some of the columns were not useful or their information was redundant:

`['transaction_country',
 'transaction_dispersion',
 'reason_code_iso',
 'reason_description',
 'reason_clasiffication',
 'merchant_country',
 'merchant_address_country',
 'transaction_business_model',
 'site_classification',
 'transaction_processing_currency',
 'isic_section_id',
 'isic_division_id',
 'ip_location_country']`
 
So, in the last notebook we did this initial selection process, adjusting the database to 34 columns, dropping those 13 columns.

In the first part of this notebook we are going to reduce the shape of the database to only what is useful, and making it a little lighter in the process. 




## Data Cleansing
The main tasks of this notebook are:
* Regular expressions pattern: 
> `transaction_payer_document_type` is a column with too many unique values, wich ones are correct?

* Cross-field validation:
> Which of the columns with country/language should we use?

* Accuracy:
> Is it possible to check for outliers and trends in the column `transaction_amount`?

* Completeness:
> is it possible to fix null values in `transaction_payer_id` with data from other rows?

* Uniformity:
> we already have take care of this in the past notebook:
 * All integers are already corrected
 * All data are already transformed
 * All float are already transformed, and all amounts have been converted to COP


The second part is about making our data more complete by adding related information. For example, appending descriptions to the abreviations and new columns that consolidate some related attributes. Data cleansing may also involve harmonization (or normalization) of data, which is the process of bringing together data of "varying file formats, naming conventions, and columns", and transforming it into one cohesive data set.

### Data Transformation
Allows the mapping of the data from its given format into the format expected by the appropriate application. This includes value conversions or translation functions, as well as normalizing numeric values to conform to minimum and maximum values.

### Duplicate Elimination
Here we require an algorithm for determining whether data contains duplicate representations of the same entity. Usually, data is sorted by a key that would bring duplicate entries closer together for faster identification.


**GOALS**

* Check for and deal with erroneous values in the dataset like typos, misspelling, upper/lowecase, spaces.
* Drop rows that contain too much missing information that are not useful
* Fill missing information
* Find Nulls and NaNs. 
  * Define What to do with Nulls and NaNs: replace with filler values, concatenate or drop.
    * Fill customer IDs
    * Check if card IDs correspond with other card columns
    * Fill merchant locations
    * Which variables have high inaccuracy or uncertainty?
* Check for and deal with erroneous values in the dataset, typos, misspelling, upper/lowecase, spaces.\
* Generate unique IDs to replace hashes
* cleaning the data as much as possible
* Transform information to the desired format


In [ ]:
#REquired libraries:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

# To load a file into colab:
import io
import os
from datetime import datetime

In [ ]:
#!pip install dovpanda
#import dovpanda

# Data Loading
First load the database, that has been already adjusted to 3 columns by 12325791 rows. 





In [ ]:
# Making lists of columns for further use:

HashedCols = ['transaction_payer_id', 'transaction_payer_email', 'IP', 'card_id']

DateCols = ['transaction_processing_date_', 'merchant_created']

IntCols = ['transaction_processing_hour', 'transaction_card_installments']

FloatCols = ['transaction_processing_amount']

CategoricalCols = DateCols + IntCols + ['transaction_request_language', 'transaction_response_code',
                                        'transaction_card_issuer_name', 'transaction_card_type', 
                                        'transaction_payer_document_type', 
                                        'paymentmethod_franchise',  'paymentmethod_name', 'paymentmethod_type',
                                        'ip_location_region_name', 'ip_location_city', 
                                        'card_class', 'card_country', 'card_issuer_country',
                                        'merchant_id', 'merchant_enabled', 
                                        'merchant_address_city', 'merchant_classification', 
                                        'isic_division_name', 'isic_section_name',
                                        'site_category', 'site_id', 'site_channel'
                                        ]

In [ ]:
All_cols = HashedCols + FloatCols + CategoricalCols + [
    'transaction_user_agent', 'transaction_id', 'transaction_description'
    ]
len(All_cols)

34

## Pickle Files


In [ ]:
pwd

'/content'

In [ ]:
cd ../DATA

/home/jovyan/work/PROJECT/DATA


In [ ]:
ls

 card_class.csv*
 CSVPreprocessing.ipynb*
 placetopayDB3.csv*
 placetopayDB3.ftr*
 placetopayDB3_pickle*
 placetopayDB4.csv*
 placetopayDB4.ftr*
 placetopayDB4_pickle*
 placetopayDB4_sample.csv*
 placetopayDB4_sampleLOG.csv*
 placetopayDB4_sampleLOG.ftr*
'Reto MinTIC - Inferencia-y-Recomendacion-personalizada.csv'*
'Reto MinTIC - Inferencia-y-Recomendacion Teams 90-67-89.csv'*
'Reto MinTIC - Inferencia-y-Recomendacion Teams 90-67-89_sample.csv'*


In [ ]:
%%time
# To save as Pickle format:
#bdf.to_pickle("./placetopayDB3_pickle")

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [ ]:
%%time
#And then when you need to restart your notebook you can simply read directly from pickle:
bd = pd.read_pickle("./placetopayDB3_pickle")
# If the file is very large the pickle file will be compressed with the parameter compression = 'gzip'.

CPU times: user 9.19 s, sys: 2.77 s, total: 12 s
Wall time: 12.7 s


#### Pickle files into Google Drive:

In [ ]:
from google.colab import drive
import pickle
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time
DATA_PATH = "/placetopayDB3_pickle"
infile = open(DATA_PATH,'rb')
bd = pickle.load(infile)

FileNotFoundError: ignored

In [ ]:
bd.shape

(11326973, 32)

# Outliers and Anomalies Detection

In [ ]:
bd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9849696 entries, 0 to 12324743
Data columns (total 32 columns):
 #   Column                           Dtype   
---  ------                           -----   
 0   transaction_user_agent           object  
 1   transaction_id                   object  
 2   transaction_description          object  
 3   transaction_processing_date_     category
 4   transaction_processing_hour      category
 5   transaction_request_language     category
 6   transaction_payer_id             object  
 7   transaction_payer_document_type  category
 8   transaction_processing_amount    float64 
 9   transaction_card_issuer_name     category
 10  transaction_card_type            category
 11  transaction_card_installments    category
 12  transaction_response_code        category
 13  site_id                          category
 14  site_channel                     category
 15  site_category                    category
 16  merchant_id                      ca

## Check for uninformative variables:

['transaction_request_language', 'transaction_card_issuer_name',         'paymentmethod_franchise',  'paymentmethod_name', 'paymentmethod_type','card_country']

In [ ]:
# check if the type of card attributes:
bd.groupby(['paymentmethod_franchise',  'paymentmethod_name', 'paymentmethod_type'])['transaction_id','transaction_processing_amount'].count().sort_values(by='transaction_id').head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


transaction_id  transaction_processing_amount
paymentmethod_franchise paymentmethod_name    paymentmethod_type                                               
DISCOVER                Discover              CREDITCARD                    18.0                           18.0
MASTERCARD              Transerver Mastercard CREDITCARD                    26.0                           26.0
VISA                    Transerver Visa       CREDITCARD                    42.0                           42.0
                        Visa Electron         CREDITCARD                  2158.0                         2158.0
DINERS                  Diners Club           CREDITCARD                 66864.0                        66864.0
AMEX                    American Express      CREDITCARD                294829.0                       294829.0
MASTERCARD              MasterCard            CREDITCARD               1722126.0                      1722126.0
VISA                    Visa                  CREDITCARD               2438725.0                      2438725.0
AMEX                    American Express      CASH                           NaN                            NaN
                                              DEBITACCOUNT                   NaN                            NaN

In [ ]:
# there are 22 different values in paymentmethod_name
[x for x in bd['paymentmethod_name'].unique()]

['Transerver Mastercard',
 'Transerver Visa',
 'MasterCard',
 'PSE',
 'Visa',
 'American Express',
 'Diners Club',
 'Corresponsales bancarios Grupo Aval',
 'Bancolombia',
 'Tarjeta Exito',
 'Visa Electron',
 'Tarjeta Alkosto',
 'Puntos GANA',
 'Tarjeta CODENSA',
 'Tarjeta CAFAM',
 'Tarjeta Tuya',
 'Discover',
 'SafetyPay',
 'Tarjeta SOMOS',
 'PayPal',
 'Efectivo Supergiros',
 'Oki']

In [ ]:
# What about the location or procedence of the payers?
bd[CategoricalCols].groupby(['card_country','transaction_request_language']).size()

card_country  transaction_request_language
CO            EN                                 7776
              ES                              4890513
              FR                                   87
              IT                                   25
              PT                                   55
EC            EN                                    6
              ES                                 4777
              FR                                    0
              IT                                    0
              PT                                    0
dtype: int64

### transaction_payer_id 

#### NI<!H%G$DCY? as a Null value

Continuing with the analisys, we found that the column 'transaction_payer_id' the register (payer) identified with the hash  **'NI<!H%G$DCY?<.=_`KMG)X6XVZ5M(>,_+*7L6U4B(DS' had 	1325459** transactions. We asked the company PayPlacetoPlay and they told us that this register is Null, but in the process of hashing Null is converted to this value. 

So, we started to research this registers and after looking to other variables as 'transaction_payer_email'and 'transaction_payer_ip' we found that most of the time the registers had also Null in the email and IP field. We cannot assert that this transactions are related to different users or to the same user in different IP, or email. We cannot say with certainity that this users are Null for any other reason. So Null doesnt have a clear meaning. 

The _transaction_payer_id_ was ```"NI<!H%G$DCY?<.=_`KMG)X6XVZ5M(>,_+*7L6U4B(DS"```

In [ ]:
len(bd[bd['transaction_payer_id']=='NI<!H%G$DCY?<.=_`KMG)X6XVZ5M(>,_+*7L6U4B(DS'])

1325459

In [ ]:
#¿How many transactions with the same problematic id there are?
bd[bd['transaction_payer_id']=='NI<!H%G$DCY?<.=_`KMG)X6XVZ5M(>,_+*7L6U4B(DS'].groupby('transaction_response_code').count()

,transaction_user_agent,transaction_id,transaction_description,transaction_processing_date_,transaction_processing_hour,transaction_request_language,transaction_payer_id,transaction_payer_document_type,transaction_payer_email,IP,transaction_processing_amount,transaction_card_issuer_name,transaction_card_type,transaction_card_installments,site_id,site_channel,site_category,merchant_id,merchant_enabled,merchant_address_city,merchant_created,merchant_classification,ip_location_region_name,ip_location_city,card_id,card_issuer_country,card_class,card_country,paymentmethod_name,paymentmethod_franchise,paymentmethod_type,isic_division_name,isic_section_name
transaction_response_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
APPROVED,477543,477543,433605,477543,477543,477543,477543,0,477543,477543,477543,476219,477543,429619,477543,477543,477543,477543,477543,477543,477543,477543,17815,17814,429619,429257,405357,429619,477543,429499,477543,477455,477455
FAILED,270,270,136,270,270,270,270,0,270,270,270,264,270,270,270,270,270,270,270,270,270,270,113,113,270,260,251,270,270,240,270,270,270
PENDING,36,36,36,36,36,36,36,0,36,36,36,36,36,36,36,36,36,36,36,36,36,36,0,0,36,32,34,36,36,22,36,36,36
REJECTED,847610,847610,840755,847610,847610,847610,847610,0,847610,847610,847610,845099,847610,847610,847610,847610,847610,847610,847610,847610,847610,847610,4695,4692,847610,835352,821135,847610,847610,804861,847610,847607,847607


In [ ]:
# How many different cards are being used by this "Null Users" (Users that have a particular hash) ?
len(bd[bd['transaction_payer_id']=='NI<!H%G$DCY?<.=_`KMG)X6XVZ5M(>,_+*7L6U4B(DS'].groupby('card_id').count())

48992

And after a conversation with the company Placetopay, they told us that the hash corresponds with the Null values in the database, so we proceed to imput a value to this hash:

In [ ]:
bd = bd.replace(["NI<!H%G$DCY?<.=_`KMG)X6XVZ5M(>,_+*7L6U4B(DS"],np.NaN)
bd.shape

(12325791, 34)

### transaction_user_agent

*   In the column ***transaction_user_agent*** the Nulls are codified as zeros


In [ ]:
bd[bd['transaction_user_agent']=='0'][All_cols].groupby(['transaction_response_code']).count()

,transaction_payer_id,transaction_payer_email,IP,card_id,transaction_processing_amount,transaction_processing_date_,merchant_created,transaction_processing_hour,transaction_card_installments,transaction_request_language,transaction_card_issuer_name,transaction_card_type,transaction_payer_document_type,paymentmethod_franchise,paymentmethod_name,paymentmethod_type,ip_location_region_name,ip_location_city,card_class,card_country,card_issuer_country,merchant_id,merchant_enabled,merchant_address_city,merchant_classification,isic_division_name,isic_section_name,site_category,site_id,site_channel,transaction_user_agent,transaction_id,transaction_description
transaction_response_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
APPROVED,1395771,908119,1834238,1641909,1834238,1834238,1834238,1834238,1641909,1834238,1828587,1834238,1395767,1440885,1834238,1834238,480265,479358,1552589,1641909,1542757,1834238,1834238,1834238,1834238,1834040,1834040,1834238,1834238,1834238,1834238,1834238,1518569
FAILED,98677,96485,98947,39362,98947,98947,98947,98947,39362,98947,98581,98947,98677,38598,98947,98947,62369,61724,36079,39362,38912,98947,98947,98947,98947,98931,98931,98947,98947,98947,98947,98947,67299
PENDING,46,24,82,80,82,82,82,82,78,82,82,82,46,50,82,82,21,21,77,80,67,82,82,82,82,82,82,82,82,82,82,82,82
REJECTED,921678,727859,1769288,1642869,1769288,1769288,1769288,1769288,1642869,1769288,1758383,1769288,921678,1468030,1769288,1769288,192404,191599,1568028,1642869,1567173,1769288,1769288,1769288,1769288,1769280,1769280,1769288,1769288,1769288,1769288,1769288,1266080


In [ ]:
bd[bd['transaction_user_agent']!='0'][All_cols].groupby(['transaction_response_code']).count()

,transaction_payer_id,transaction_payer_email,IP,card_id,transaction_processing_amount,transaction_processing_date_,merchant_created,transaction_processing_hour,transaction_card_installments,transaction_request_language,transaction_card_issuer_name,transaction_card_type,transaction_payer_document_type,paymentmethod_franchise,paymentmethod_name,paymentmethod_type,ip_location_region_name,ip_location_city,card_class,card_country,card_issuer_country,merchant_id,merchant_enabled,merchant_address_city,merchant_classification,isic_division_name,isic_section_name,site_category,site_id,site_channel,transaction_user_agent,transaction_id,transaction_description
transaction_response_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
APPROVED,6602860,6602860,6641936,1164291,6641936,6641936,6641936,6641936,1164286,6641936,6636727,6641936,6602860,1163991,6641936,6641936,6339671,6324239,1102785,1164291,1162966,6641936,6641936,6641936,6641936,6641845,6641845,6641936,6641936,6641936,6641936,6641936,6562978
FAILED,86527,86527,86527,7568,86527,86527,86527,86527,7568,86527,86466,86527,86527,7565,86527,86527,82720,82621,7066,7568,7550,86527,86527,86527,86527,86527,86527,86527,86527,86527,86527,86527,85625
PENDING,40,40,40,15,40,40,40,40,13,40,40,40,40,15,40,40,19,18,13,15,15,40,40,40,40,36,36,40,40,40,40,40,40
REJECTED,1894733,1894733,1894733,407145,1894733,1894733,1894733,1894733,407141,1894733,1886817,1894733,1894733,405654,1894733,1894733,1812216,1794549,378226,407145,405177,1894733,1894733,1894733,1894733,1894675,1894675,1894733,1894733,1894733,1894733,1894733,1877151


In [ ]:
# Replace "null values" for np.NaN values
bd['transaction_user_agent'] = bd['transaction_user_agent'].replace(["0"],np.NaN)
bd.shape

(12325791, 34)

#### Test transactions


> As the entity told us there're some ***test transactions*** and since this ones doesn't give us any info we can drop them cause they aren't too many.

In [ ]:
bd[bd['transaction_description'].str.contains(r'prueba',case=False, na=False)].head()

,transaction_user_agent,transaction_id,transaction_description,transaction_processing_date_,transaction_processing_hour,transaction_request_language,transaction_payer_id,transaction_payer_document_type,transaction_payer_email,IP,transaction_processing_amount,transaction_card_issuer_name,transaction_card_type,transaction_card_installments,transaction_response_code,site_id,site_channel,site_category,merchant_id,merchant_enabled,merchant_address_city,merchant_created,merchant_classification,ip_location_region_name,ip_location_city,card_id,card_issuer_country,card_class,card_country,paymentmethod_name,paymentmethod_franchise,paymentmethod_type,isic_division_name,isic_section_name
32,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,COA1485595049,Prueba,2020-03-31,12,ES,"K[`JBNB5$Y?Y!]*&Q""/S*&V0N4/';@0^]);6D\X#7I-",CC,F[/>W+UKJZQU.*+G)0DX-^33IT$B]'CQV4NLKB+%O0+,"D\A""`H@+T#6AS%G6HT(M]$.D=Q8%K_O^#61?``LW5>`]$....",3821.7,Bancolombia,C,1.0,APPROVED,CO0000005835,INTERNET,E-Commerce,CO0000001632,Activo,San Jose,2020-04-03,N,CO-Antioquia,Envigado,2EECA22FFFC3D1A2F53953563B7925FAE4F8FD56,CO,ESTANDAR,EC,Transerver Mastercard,MASTERCARD,CREDITCARD,NaN,NaN
38,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,COA1485595049,Prueba,2020-03-31,12,ES,"K[`JBNB5$Y?Y!]*&Q""/S*&V0N4/';@0^]);6D\X#7I-",CC,F[/>W+UKJZQU.*+G)0DX-^33IT$B]'CQV4NLKB+%O0+,"D\A""`H@+T#6AS%G6HT(M]$.D=Q8%K_O^#61?``LW5>`]$....",3821.7,Bancolombia,C,1.0,APPROVED,CO0000005835,INTERNET,E-Commerce,CO0000001632,Activo,San Jose,2020-04-03,N,CO-Antioquia,Envigado,2EECA22FFFC3D1A2F53953563B7925FAE4F8FD56,CO,ESTANDAR,CO,Transerver Mastercard,MASTERCARD,CREDITCARD,NaN,NaN
487,Mozilla/5.0 (Linux; Android 8.1.0; BLACK_PRO) ...,COA1490301937,Pago básico de prueba Royal Films,2020-09-07,23,ES,B7-$M-1<5T:VOZCH[;/B(K#8LR5B0\`6>S($MZ^N2)1,CC,CD_Y=22[Q0$1$6:5(@%X]Q&EO?-#W4I*?LAWW43GMH(,D3M%3I)LU#6Q.HC[C:6`]$.LC@6W3Y2K#6)&3F^A!S\]$....,40000.0,Scotiabank Colpatria,D,NaN,APPROVED,CO0000005080,INTERNET,E-Commerce,CO0000001437,Activo,Barranquilla,2019-03-01,C,CO-Bogota D.C.,Bogota,NaN,NaN,NaN,NaN,PSE,NaN,DEBITACCOUNT,59.0-Actividades cinematográficas,J-INFORMACIÓN Y COMUNICACIONES
2947,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,COA1486468086,Pago reserva 75491 [PRUEBAS],2020-04-23,9,ES,"KPFHOM5133/^+TPKT0""Z0&?&30\0T`&P))<+9""F*792",TI,"KI823REY5AE(O""""T&R(!;E[<@MI[.'[YCB;E0DKR##M","PLRI+XO!0$6-#""1G`FPN]$.L74B]@%OM$6)EBK;Z*C\]$....",52493.0,Bancolombia,C,1.0,REJECTED,CO0000005621,INTERNET,E-Commerce,CO0000001131,Activo,Medellin,2017-05-25,C,CO-Antioquia,Itaguei,19ECCD4904C6E912007E98C74BB42D119871072C,CO,PLATINUM,CO,MasterCard,MASTERCARD,CREDITCARD,79.0-Agencias de viajes y operadores turísticos,N-ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y D...
5782,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,COA1491287183,Compra de recargas website-PayU-PRUEBAS,2020-07-27,12,ES,"A@DC@^)?8LPGA!-\5CGC_+G-X.GTE.""$';I2YL;XQ$D",CC,"O3)RT(,.C^29N7\`B.15!!""\?9LM?[P#H[,>2>PB<X2","LB\E3;Z""1$69,>?Q\17\]$.D(%""]N482$6A4VAY/K8M]$....",9000.0,Scotiabank Colpatria,D,NaN,REJECTED,CO0000003048,INTERNET,E-Commerce,CO0000000825,Activo,Bogota,2014-09-07,A+,CO-Bogota D.C.,Bogota,NaN,NaN,NaN,NaN,PSE,NaN,DEBITACCOUNT,61.0-Telecomunicaciones,J-INFORMACIÓN Y COMUNICACIONES


In [ ]:
PruebasDF = bd[((bd['transaction_description'].str.contains(r'^(?!.*saber).*prueba.*$',case=False).fillna(False)))]
PruebasDF

,transaction_user_agent,transaction_id,transaction_description,transaction_processing_date_,transaction_processing_hour,transaction_request_language,transaction_payer_id,transaction_payer_document_type,transaction_payer_email,IP,...,ip_location_city,card_id,card_issuer_country,card_class,card_country,paymentmethod_name,paymentmethod_franchise,paymentmethod_type,isic_division_name,isic_section_name
32,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,COA1485595049,Prueba,2020-03-31,12,ES,"K[`JBNB5$Y?Y!]*&Q""/S*&V0N4/';@0^]);6D\X#7I-",CC,F[/>W+UKJZQU.*+G)0DX-^33IT$B]'CQV4NLKB+%O0+,"D\A""`H@+T#6AS%G6HT(M]$.D=Q8%K_O^#61?``LW5>`]$....",...,Envigado,2EECA22FFFC3D1A2F53953563B7925FAE4F8FD56,CO,ESTANDAR,EC,Transerver Mastercard,MASTERCARD,CREDITCARD,NaN,NaN
38,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,COA1485595049,Prueba,2020-03-31,12,ES,"K[`JBNB5$Y?Y!]*&Q""/S*&V0N4/';@0^]);6D\X#7I-",CC,F[/>W+UKJZQU.*+G)0DX-^33IT$B]'CQV4NLKB+%O0+,"D\A""`H@+T#6AS%G6HT(M]$.D=Q8%K_O^#61?``LW5>`]$....",...,Envigado,2EECA22FFFC3D1A2F53953563B7925FAE4F8FD56,CO,ESTANDAR,CO,Transerver Mastercard,MASTERCARD,CREDITCARD,NaN,NaN
487,Mozilla/5.0 (Linux; Android 8.1.0; BLACK_PRO) ...,COA1490301937,Pago básico de prueba Royal Films,2020-09-07,23,ES,B7-$M-1<5T:VOZCH[;/B(K#8LR5B0\`6>S($MZ^N2)1,CC,CD_Y=22[Q0$1$6:5(@%X]Q&EO?-#W4I*?LAWW43GMH(,D3M%3I)LU#6Q.HC[C:6`]$.LC@6W3Y2K#6)&3F^A!S\]$....,...,Bogota,NaN,NaN,NaN,NaN,PSE,NaN,DEBITACCOUNT,59.0-Actividades cinematográficas,J-INFORMACIÓN Y COMUNICACIONES
2947,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,COA1486468086,Pago reserva 75491 [PRUEBAS],2020-04-23,9,ES,"KPFHOM5133/^+TPKT0""Z0&?&30\0T`&P))<+9""F*792",TI,"KI823REY5AE(O""""T&R(!;E[<@MI[.'[YCB;E0DKR##M","PLRI+XO!0$6-#""1G`FPN]$.L74B]@%OM$6)EBK;Z*C\]$....",...,Itaguei,19ECCD4904C6E912007E98C74BB42D119871072C,CO,PLATINUM,CO,MasterCard,MASTERCARD,CREDITCARD,79.0-Agencias de viajes y operadores turísticos,N-ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y D...
5782,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,COA1491287183,Compra de recargas website-PayU-PRUEBAS,2020-07-27,12,ES,"A@DC@^)?8LPGA!-\5CGC_+G-X.GTE.""$';I2YL;XQ$D",CC,"O3)RT(,.C^29N7\`B.15!!""\?9LM?[P#H[,>2>PB<X2","LB\E3;Z""1$69,>?Q\17\]$.D(%""]N482$6A4VAY/K8M]$....",...,Bogota,NaN,NaN,NaN,NaN,PSE,NaN,DEBITACCOUNT,61.0-Telecomunicaciones,J-INFORMACIÓN Y COMUNICACIONES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12302183,Mozilla/5.0 (iPhone; CPU iPhone OS 13_5 like M...,COA1491134544,Compra de recargas website-PayU-PRUEBAS,2020-07-24,8,ES,"LK6ZI?3O9N[\83""!9PC!=-CAG+L\K->-.L^,I;0I-K>",CC,"DGK25>73)M*+4R,;5;@I;X]FYK$ZP]R3J'?\>U^U;HE",H&O1[86S3#65:.V/'.\Q]$.L(7Z.+4@_$6Y_'M+'Q.]]$....,...,Medellin,NaN,NaN,NaN,NaN,PSE,NaN,DEBITACCOUNT,61-Telecomunicaciones,J-INFORMACIÓN Y COMUNICACIONES
12302234,Mozilla/5.0 (Linux; Android 10; SM-J600G) Appl...,COA1491229708,Compra de recargas website-PayU-PRUEBAS,2020-07-26,10,ES,"DNB)J'FVI]Y,]G3B[9)333OTX<AQ`QKR?EC1J-T914F",CC,"J+8E[#=\0N@9K?T865""\6_,B?E2R5V*D:$][9^0(@!""","PLRI+XO!0$6-#""1G`FPN]$.LS!O)BIQS$6)2L=OHB9I]$....",...,Medellin,NaN,NaN,NaN,NaN,PSE,NaN,DEBITACCOUNT,61-Telecomunicaciones,J-INFORMACIÓN Y COMUNICACIONES
12302235,Mozilla/5.0 (Linux; Android 9; MAR-LX3A) Apple...,COA1491225990,Compra de recargas website-PayU-PRUEBAS,2020-07-25,22,ES,"G9['<,)<Q.E/@^89D\V'HUL""[PBW)&V[A&YM&&OI_%W",CC,"GO^1FS.5S^-]'""""/#F:!;A`R0(20`+D-AO&Q?O:FF3P","HOC.H8M22#6%LUK^Y_DK]$.L1RZ3;*9,#6I9SX1V7VX]$....",...,Medellin,NaN,NaN,NaN,NaN,PSE,NaN,DEBITACCOUNT,61-Telecomunicaciones,J-INFORMACIÓN Y COMUNICACIONES
12302248,Mozilla/5.0 (Linux; Android 10; SNE-LX3) Apple...,COA1491142365,Compra de recargas website-PayU-PRUEBAS,2020-07-24,10,ES,B.2VU&/>S5NS]XS8W=:V'0YIWM8&9)P]G.?^UCG_935,CC,D_@_O(PC?IJ6=)^RB9LZ*2N>01ERI>A411>4-<]5Y06,"PLRI+XO!0$6-#""1G`FPN]$.PN166\M*E$6M3:N84BXN]$....",...,Medellin,NaN,NaN,NaN,NaN,PSE,NaN,DEBITACCOUNT,61-Telecomunicaciones,J-INFORMACIÓN Y COMUNICACIONES


In [ ]:
len(PruebasDF)
#bd['transaction_description'].iloc[280596]
#[x for x in PruebasDF if bd['transaction_description'].iloc[x]]

8308

In [ ]:
# how many test transactions 'prueba' are rejected or failed?
PruebasDF.groupby('transaction_response_code').count()

,transaction_user_agent,transaction_id,transaction_description,transaction_processing_date_,transaction_processing_hour,transaction_request_language,transaction_payer_id,transaction_payer_document_type,transaction_payer_email,IP,transaction_processing_amount,transaction_card_issuer_name,transaction_card_type,transaction_card_installments,site_id,site_channel,site_category,merchant_id,merchant_enabled,merchant_address_city,merchant_created,merchant_classification,ip_location_region_name,ip_location_city,card_id,card_issuer_country,card_class,card_country,paymentmethod_name,paymentmethod_franchise,paymentmethod_type,isic_division_name,isic_section_name
transaction_response_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
APPROVED,3986,4418,4418,4418,4418,4418,4416,4416,4412,4418,4418,4418,4418,844,4418,4418,4418,4418,4418,4418,4418,4418,4248,4246,844,842,813,844,4418,827,4418,4412,4412
FAILED,65,158,158,158,158,158,158,158,158,158,158,158,158,57,158,158,158,158,158,158,158,158,157,157,57,57,57,57,158,56,158,158,158
PENDING,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,0,1,1,1
REJECTED,3392,3731,3731,3731,3731,3731,3729,3729,3729,3731,3731,3731,3731,399,3731,3731,3731,3731,3731,3731,3731,3731,3520,3521,399,396,348,399,3731,371,3731,3730,3730


In [ ]:
# Dropping all test and 'prueba' rows (except 'Prueba Saber Pro')
bd.drop(PruebasDF.index, axis = 0, inplace =True)

In [ ]:
# Drop all transactions that contain 'test' in the description
bd.drop(bd[bd['transaction_description'].str.contains('Test',case=False).fillna(False)].index, axis = 0, inplace =True)

In [ ]:
len(bd[bd['transaction_card_issuer_name']=='Banco De Pruebas'])

3563

In [ ]:
# Dropping all 'banco de prueba' rows
bd.drop(bd[bd['transaction_card_issuer_name']=='Banco De Pruebas'].index, axis = 0, inplace =True)

In [ ]:
# Dropping all 'banco de prueba' rows
bd.drop(bd[bd['transaction_card_issuer_name'].str.contains(r'prueba',case=False, na=False)].index, axis = 0, inplace =True)
bd.shape

In [ ]:
len(bd[bd['transaction_description'].str.contains(r'prueba',case=False, na=False)])

409

### Transaction_payer email and IP


In [ ]:
# How many emails can be extracted from transactions without payer_id?
bd[bd['transaction_payer_id'].isna()]['transaction_payer_email'].value_counts()

J"R\/F.JX;J58&W)*2'S+)&-#V1=Y4LTX#QYN;N484=    416
P(_+1YMXH75I+7%6K?.]JO3G1"B#7@:$;B(G_=K&I@/    284
P^;ID@:@685)I"T:R2SZJO+\7\DH.0>$;(.C[E_47?/    195
H/OP>ZK2T+VR=ICG!'A76(,%KB_@EPG,08P+7]\02)>    159
PL^P0Z;O\75IV<W`O=SUJO#L_%[@`V@$;6__3!8PS@/     72
                                              ... 
K9V73%*FQ!9JOS0](IK:%,)]LDVG:9%O.;S`$#[U'1B      1
D9U:'6?N?894'3DN$V!NG,,&$]Y!`<G7,OUC&'DBUMR      1
HURW'5G^;;EWYZ(`,%?^[(UFT%^!ROOZC_7SQ&\-`+Z      1
LRG*T6"#]""OJ/:Z&JL+SS[1Q!2[S-2Y;EC9+5D@!R8      1
JPYR^AQS.&R6(0[>,](FM\#(<TMV(CV+.RU/&*T@%DD      1
Name: transaction_payer_email, Length: 3016, dtype: int64

In [ ]:
# How many cards can be extracted from transactions without payer_id?
bd[bd['transaction_payer_id'].isna()]['card_id'].value_counts()

E0998702EA4730485DA519342BFA9692BB054D56    1571
60EE9EAE588C474988514C76DB1970E7CCE61BD1    1411
C18483B10D1E06F5ADA6A47D5E6F91F967E722F6    1253
956436FB91275DB7982DA68711628B8C1DD8055D    1047
48C21431739B2C78139B723BDABF604873FF8B46     979
                                            ... 
056DE3711EDB539476FD94DC3967C7F83AECE36A       1
35382330DFE4EE84CD28215C0645D68B6219A8D5       1
CBE9E14E6718619E77B43218064DE0FD06018A76       1
E6EC391FE775331A49B582B87C752D6A752295E8       1
08D0C2A2AA07A659CFA09F256345F291E891B465       1
Name: card_id, Length: 48988, dtype: int64

### transaction_description

> If we want to extract information about the product from the ***transaction_description*** we gotta be aware of this names that appear on this column

In [ ]:
# some of the transaction descriptions contain names and Id numbers of customers.
bd[bd['transaction_description'].str.contains('(cc) (\d{8,11})',regex=True).fillna(False)]['transaction_description'].value_counts().head()

NELSON ABRAHAM CARDENAS ESTRADA cc 79453077     6
DEIFAN BEATRIZ QUINTERO VALVERDE cc 29282858    4
consulta hematologia cc 19122324                2
Richard Jhon Ospina cc 71774175                 2
consulta hematologia cc 35467264                2
Name: transaction_description, dtype: int64

In [ ]:
# How many transactions without description are rejected or failed?
bd[bd['transaction_description'].isna()].groupby('transaction_response_code').count()

,transaction_user_agent,transaction_id,transaction_description,transaction_processing_date_,transaction_processing_hour,transaction_request_language,transaction_payer_id,transaction_payer_document_type,transaction_payer_email,IP,transaction_processing_amount,transaction_card_issuer_name,transaction_card_type,transaction_card_installments,site_id,site_channel,site_category,merchant_id,merchant_enabled,merchant_address_city,merchant_created,merchant_classification,ip_location_region_name,ip_location_city,card_id,card_issuer_country,card_class,card_country,paymentmethod_name,paymentmethod_franchise,paymentmethod_type,isic_division_name,isic_section_name
transaction_response_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
APPROVED,78958,394627,0,394627,394627,394627,350689,350685,344617,394627,394627,392555,394627,325405,394627,394627,394627,394627,394627,394627,394627,394627,93675,93617,325405,322695,309535,325405,394627,321799,394627,394597,394597
FAILED,902,32550,0,32550,32550,32550,32416,32416,32417,32550,32550,32242,32550,31256,32550,32550,32550,32550,32550,32550,32550,32550,1837,1837,31256,31181,28439,31256,32550,31245,32550,32550,32550
PENDING,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
REJECTED,17537,520481,0,520481,520481,520481,513626,513626,513234,520481,520481,514156,520481,507036,520481,520481,520481,520481,520481,520481,520481,520481,35061,35047,507036,504407,473908,507036,520481,503917,520481,520480,520480


# Remove Duplicates

In [ ]:
bd['transaction_id'].value_counts().head(20)

COA1492230455    2
COA1486305216    2
COA1491508658    2
COA1486732617    2
COA1483403490    2
COA1483634393    2
COA1481871618    2
COA1482316473    2
COA1492640062    2
COA1493690830    2
COA1487583203    2
COA1488693804    2
COA1492751918    2
COA1481791404    2
COA1484648111    2
COA1493240163    2
COA1482309695    2
COA1484166413    2
COA1482331144    2
COA1483224911    2
Name: transaction_id, dtype: int64

In [ ]:
# just to check, the difference between duplicate transactions is the 'card_country'
bd[bd['transaction_id']=='COA1484911924']


,transaction_user_agent,transaction_id,transaction_description,transaction_processing_date_,transaction_processing_hour,transaction_request_language,transaction_payer_id,transaction_payer_document_type,transaction_payer_email,IP,transaction_processing_amount,transaction_card_issuer_name,transaction_card_type,transaction_card_installments,transaction_response_code,site_id,site_channel,site_category,merchant_id,merchant_enabled,merchant_address_city,merchant_created,merchant_classification,ip_location_region_name,ip_location_city,card_id,card_issuer_country,card_class,card_country,paymentmethod_name,paymentmethod_franchise,paymentmethod_type,isic_division_name,isic_section_name
6167042,NaN,COA1484911924,005618016-000000000000,2020-03-13,9,ES,NaN,NaN,NaN,NI<!H%G$DCY?<.=_`KMG)..NI<!H%G$DCY?<.=_`KMG).....,916000.0,Bbva Colombia S.A,C,1.0,REJECTED,CO0000002752,POS,Recaudo,CO0000000996,Activo,Bogota,2016-01-06,A+,NaN,NaN,9E65A087A978405110285799758765AB4BC5CF08,CO,BLACK CARD,EC,MasterCard,MASTERCARD,CREDITCARD,65-Seguros y fondos de pensiones,K-ACTIVIDADES FINANCIERAS Y DE SEGUROS
10597692,NaN,COA1484911924,005618016-000000000000,2020-03-13,9,ES,NaN,NaN,NaN,NI<!H%G$DCY?<.=_`KMG)..NI<!H%G$DCY?<.=_`KMG).....,916000.0,Bbva Colombia S.A,C,1.0,REJECTED,CO0000002752,POS,Recaudo,CO0000000996,Activo,Bogota,2016-01-06,A+,NaN,NaN,9E65A087A978405110285799758765AB4BC5CF08,CO,BLACK CARD,CO,MasterCard,MASTERCARD,CREDITCARD,65-Seguros y fondos de pensiones,K-ACTIVIDADES FINANCIERAS Y DE SEGUROS


In [ ]:
# Remove Duplicates:
bd = bd.drop_duplicates(subset=['transaction_id'], keep='first')
bd.shape

(11326973, 32)

# Data Transformation

## payer_document_type

we can look at the transaction_payer_document_type, to correct the categories:

In [ ]:
bd['transaction_payer_document_type'].isna().sum()

1324487

In [ ]:
len(bd[bd['transaction_payer_document_type'].notnull()])

10986204

In [ ]:
%%time
# DONT RUN We are having this column as categorical, we need to convert to string. there are two ways:
#bd['transaction_payer_document_type'] = bd['transaction_payer_document_type'].astype(str)

#Or, inpute NaNs as an empty string, before applying regex:
#bd['transaction_payer_document_type'].fillna('nan', inplace=True)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.06 µs


In [ ]:
documents = bd['transaction_payer_document_type'].copy()
len(documents)

12310691

In [ ]:
documents.value_counts().head(20)
#documents.value_counts().index.tolist()

CC            9923316
NIT            803802
TI              88218
CE              83559
PPN             43741
Otro            17113
LIC              1359
SSN              1213
TAX               955
RUT               675
860034313         634
CIP               458
CI                424
52968953          383
26023121          364
860002503         355
DIE               315
CPF               283
1014187091        232
860026753         131
Name: transaction_payer_document_type, dtype: int64

In [ ]:
# we can use this list 
len(list(documents.unique()))

6786

In [ ]:
duplicates# Importing re package for using regular expressions 
import re 
  
# Function to clean the document types 
def Clean_names(Name): 
  Name = str(Name)
  # Search for opening bracket in the name followed by 
  # any characters repeated any number of times 
  if re.search(r'\d.*', Name):
      # Extract the position of beginning of pattern 
      pos = re.search(r'\d.*', Name).start() 
      if re.search(r'\d{8,11}', Name):  # we assume any string of digits from 8 to 11 long is a cedula
        return 'CC'
      elif pos == 0:
        return 'Otro'
      else:
        Name = Name[:pos]
        return Clean_names(Name)
        #return the cleaned name 
  elif re.search(r'PP', Name):  # We assume pp is a type of passport
    return 'PP'
  elif re.search(r'null|NONE|nan'  ,Name):
    return 'nan'
  elif re.search(r'RUT|RUC|NIT', Name): # we assume any of these is a company tax ID
    return 'NIT'
  elif re.search(r'.*CPF.*', Name): #CPF is the ID document for physical persons in Brazil
    return 'CPF'
  elif re.search(r'.*CPJ.*', Name): #CPJ/CNPJ is the ID document for business persons in Brazil
    return 'CPJ'
  elif re.search(r'Citizenship', Name):
    return 'Otro'
  elif re.search(r'.*CC.*', Name):  # We assume this is always a cedula
    return 'CC'
  elif re.search(r'.*CE.*', Name):  # We assume this is cedula of alien individual
    return 'CE'
  elif re.search(r'.*CI.*', Name): # it can be 'Cartao de identidade' in brazil
    return 'CIP'
  #elif re.search(r'\bC\b', Name):  #This is no longuer necessary
    #return 'CC'
  elif re.search(r'TI', Name):  # We assume this is 'Tarjeta de Identidad' for colombian underage individuals
    return 'TI'
  elif re.search(r'LIC|SSN|TAX', Name): # We assume this is Drivers Licence, Social Security Number or Tax ID, in USA.
    return Name                         # If is TaxID, we can say that this is a business, but with no certainty.
  # if clean up needed return the same name 
  else:
    return 'Otro'

In [ ]:
# Check the function
print([(x, Clean_names(x)) for x in documents.value_counts().index.tolist()])

[('CC', 'CC'), ('NIT', 'NIT'), ('TI', 'TI'), ('CE', 'CE'), ('PPN', 'PP'), ('Otro', 'Otro'), ('LIC', 'LIC'), ('SSN', 'SSN'), ('TAX', 'TAX'), ('RUT', 'NIT'), ('860034313', 'CC'), ('CIP', 'CIP'), ('CI', 'CIP'), ('52968953', 'CC'), ('26023121', 'CC'), ('860002503', 'CC'), ('DIE', 'Otro'), ('CPF', 'CPF'), ('1014187091', 'CC'), ('860026753', 'CC'), ('1067849768', 'CC'), ('860002180', 'CC'), ('830035702', 'CC'), ('860032463', 'CC'), ('1042365897', 'CC'), ('1032508609', 'CC'), ('28558108', 'CC'), ('1109875680', 'CC'), ('TE', 'Otro'), ('860013798', 'CC'), ('1070627669', 'CC'), ('71791325', 'CC'), ('1018441809', 'CC'), ('1032497824', 'CC'), ('1077875621', 'CC'), ('1014240846', 'CC'), ('CRCPF', 'CPF'), ('9794946', 'Otro'), ('1014389257', 'CC'), ('53001117', 'CC'), ('1020783229', 'CC'), ('1032422461', 'CC'), ('1014231996', 'CC'), ('DNI', 'Otro'), ('RUC', 'NIT'), ('1007316764', 'CC'), ('PP', 'PP'), ('1019008512', 'CC'), ('80239120', 'CC'), ('80760990', 'CC'), ('1019025545', 'CC'), ('1000283657', 'C

In [ ]:
#Check the function          
documents_new = documents.apply(Clean_names)
documents_new.value_counts().head(20)

CC      9943101
NIT      804511
TI        88218
CE        83606
PP        43907
Otro      18095
LIC        1359
SSN        1213
TAX         955
CIP         882
CPF         322
nan          24
CPJ          11
Name: transaction_payer_document_type, dtype: int64

In [ ]:
%%time
# Updated the db column 
bd['transaction_payer_document_type'] = bd['transaction_payer_document_type'].apply(Clean_names)

In [ ]:
len(bd[bd['transaction_payer_document_type'].notnull()])

10986204

In [ ]:
bd['transaction_payer_document_type'].value_counts().head(20)

In [ ]:
bd['transaction_payer_document_type'] = bd['transaction_payer_document_type'].replace("nan",np.NaN)

In [ ]:
bd['transaction_payer_document_type'].isna().sum()

1324511

In [ ]:
bd['transaction_payer_document_type'] = bd['transaction_payer_document_type'].astype('category')

## Card_class

In [ ]:
bd['card_class'].value_counts().head(20)

In [ ]:
bd[bd['card_class'].str.contains(r'gold',case=False, na=False)]['card_class'].value_counts()

In [ ]:
# replace one by one manually:
bd['card_class'] = bd['card_class'].str.replace('GOLD - Lan','Lan GOLD')

In [ ]:
cardclasses = bd['card_class'].copy()

In [ ]:
# WARNING: This is a test function.
# The function has to correct upper/lowercases and different spellings or synonims. This has to be improved
#
# Even then, there is an informed decition to make, for example:
# are all the 'GOLD' cards the same type?
# or there is a significant difference when they are issued by 'Studio-F' or 'Lan' as E-Cards?

# Function to clean the document types 

def Clean_cards(Name): 
    Name = str(Name)
    if re.search(r'GOLD|gold|Gold|Oro|ORO', Name):  # We assume pp is a type of passport
        return 'GOLD'
    elif re.search(r'CLASSIC|Clásica|Clasic'  ,Name):
        return 'CLASSIC'
    elif re.search(r'DEBIT STANDARD', Name):  # 
        return 'DEBIT STANDARD'
    elif re.search(r'STANDARD|Standard|estandar|ESTANDAR', Name): #
        return 'STANDARD'
    elif re.search(r'PLATINUM|Platinum', Name): #
        return 'PLATINUM'
    elif re.search(r'E-CARD E-PREPAGO', Name): #
        return 'E-CARD E-PREPAGO'
    elif re.search(r'MASTER DEBIT', Name):  # 
        return 'MASTER DEBIT'
    elif re.search(r'DEBIT|Debito', Name):  # 
        return 'DEBIT'
    elif re.search(r'BUSINESS', Name):  # 
        return 'BUSINESS'
    elif re.search(r'CORPORATE', Name):  # 
        return 'CORPORATE'
    elif re.search(r'INFINITE|Infinite', Name):  #
        return 'INFINITE'
    elif re.search(r'SIGNATURE|Signature', Name): #
        return 'SIGNATURE'
    elif re.search(r'ELECTRON|Electron', Name):  #
        return 'ELECTRON'
    elif re.search(r'GREEN', Name):  # 
        return 'GREEN'
    elif re.search(r'BLUE', Name):
        return 'BLUE'
    elif re.search(r'BLACK|Black', Name):  #
        return 'BLACK'
    elif re.search(r'other terms.... ', Name): #
        return Name                       # 
    
    # if clean up finished and there was no match, return the same name 
    
    else:
        return Name # or return 'Other'

In [ ]:
cardclasses = cardclasses.apply(Clean_cards)

In [ ]:
cardclasses.value_counts().head(30)

In [ ]:
%%time
# Updated the db column 
#bd['card_class'] = bd['card_class'].apply(Clean_cards)
#bd['card_class'].value_counts().head(20)

CPU times: user 749 ms, sys: 0 ns, total: 749 ms
Wall time: 747 ms


In [ ]:
# How many cards indicate a 'Test' transaction?
len(bd[bd['card_class']=='PRUEBAS'])
bd.drop(bd[bd['card_class']=='PRUEBAS'].index, axis = 0, inplace =True)

In [ ]:
bd.card_class.isna().sum()

CardsColumn = pd.DataFrame(bd.card_class.copy())

#CardsColumn.card_class.unique()

#CardsColumn.value_counts().head(20)

CardDict = pd.read_csv('card_class.csv', sep = ';')
#CardDict

CardsColumn = pd.merge(CardsColumn, CardDict, how="left", left_on='card_class',right_on='card_class')
#CardsColumn.CardNAME.value_counts().head(30)

#CardsColumn.CardNAME.isna().sum()

bd['card_class'] = CardsColumn.CardNAME

Otro    4342555
TI       295196
NIT        1545
CE          690
CC           15
CIP           1
Name: card_class, dtype: int64

## transaction_processing_amount 
we converted the type of the transaction processing amount column into float to get numerical values and be able to compute accurate graphics or tables

In [ ]:
# DONT RUN
#bd['transaction_processing_amount'] = bd['transaction_processing_amount'].str.replace(',','.').astype('float64')
type(bd['transaction_processing_amount'][0])

numpy.float64

In [ ]:
# WHat happened with the transactions with low value?
bd[bd['transaction_processing_amount']==0].describe() #there are 342 transactions for $ 0
bd[bd['transaction_processing_amount']<=1000].describe()

,transaction_processing_amount
count,37782.000000
mean,732.187436
std,314.871951
min,0.000000
25%,693.000000
50%,833.000000
75%,1000.000000
max,1000.000000


# Data Enhancement

We can create some new columns. For example, a column that marks if a card or a payer_id are possibly companies and not infividuals. This is done selecting the outliers and crossing that information between the variables # of transactions and $ of transactions. 

In [ ]:
%%time
# How many payer IDs are associated with a single card_id? Why so many?
# we still dont know, but we suspect that there are some aggregators that make purchases in the name of various payer_ids
CardsDF = bd[['transaction_processing_amount', 'transaction_payer_id', 'card_id']].groupby(['card_id','transaction_payer_id']).count().rename(columns={'transaction_processing_amount':'No_of_transactions'})

CPU times: user 22.4 s, sys: 786 ms, total: 23.2 s
Wall time: 24.1 s


In [ ]:
CardsDF.sort_values(by='No_of_transactions', ascending=False)

,,No_of_transactions
card_id,transaction_payer_id,
1F3A9DB22EDD43E62FF900BDEE0F0040090C751A,"IG`6'FM+;RN%)G`)(,+)7]Q_S;B=`*]_(H^`[""TL$Z_",2665
2468E2D261CCF660283B7FF741D49783DCF63EA1,"PBUH,R%3.`\\R56:1\:ES6DY>3G8&IN)7A<<W]WQR2_",2334
BCFE9DBE57229E7DF92BCE6B48754CCAF8212C23,"H<J[#+`SA`H4_ND&<73^:B4&B^509GE=BRZ*_F+,!AU",1175
A6B8AED46127A3D1643E91B4844DECBF575874A7,"OVCEL@[!LRF`U^]=N""7AC-;$+?<FE(8K8>!H""&8[&5#",1044
A1DFFA01709B1F1AEA5CB5AF8514917BC283B4FF,"K!<C.UBC%-B?EPD[G`N(EWVK.LTTW&H""HFQ3""@?P9SX",1009
...,...,...
395CE1B0DA5636AAFBEF1277F8A32083A6DA1751,"KY9<A.2.7%1\6:`(T*,R\H?5'F`M@KE#&]W;RA3M@*?",1
395CC859B8EAEA3B7454129300D66B7D1B8B129E,"L>W11662DI""Y(57?P>F,[=8M-$7#J;E/5+""@#VM1HJ@",1
92D2A10115A9282F460F4A02E6D61A009D7DFB99,"N,W)%U&P""\DB$)T_&#LG/*Q7%[2'J""JP9*T2N,+T3\=",1


In [ ]:
#This is the number of transactions made with the same card
CardsDF.reset_index().describe()

,No_of_transactions
count,1.393773e+06
mean,2.596401e+00
std,7.287262e+00
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,2.000000e+00
max,2.665000e+03


In [ ]:
# So we make another dataframe to count the number of payer_ids associated with a single card
CardsDf2 = CardsDF.reset_index().groupby(['card_id']).count()
CardsDf2 = CardsDf2.drop('transaction_payer_id', axis = 1).rename(columns={'No_of_transactions':'No_of_payer_ids'})
CardsDf2.describe()

,No_of_payer_ids
count,1.256703e+06
mean,1.109071e+00
std,7.754173e-01
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,1.000000e+00
max,2.940000e+02


In [ ]:
# In order to trust 'card_id' as an identifier of a user. Maybe we can drop all the cards that make purchases in name of more than 4 users:
CardsDf2.quantile([0.95, 0.99, 0.999, 0.9999])

,No_of_payer_ids
0.9500,2.0
0.9900,3.0
0.9990,5.0
0.9999,15.0


In [ ]:
HighIDsCards = CardsDf2[CardsDf2['No_of_payer_ids']>=4]
HighIDsCards.sort_values(by='No_of_payer_ids')

,No_of_payer_ids
card_id,
FFFBE715D3F1F82139F7091516E4AE0690E7FA96,4
6DBD3584BCAA60DF65F47EF36F110CA97BCE8234,4
6DBEB30F012AB754DBDDDCE972A6E5AB55412CDC,4
6DCB3CCCD8EBDC2724CFBCC114FD2D4143513AF9,4
C4B4C08B04F2AEDCA5C795E80C82D8ADEC2CA899,4
...,...
B5585D38C868F13DBD12F2A940CEF41C1CA6F448,170
A725E8D5601B0D228055C88E24AE55EB1743B15A,197
5E95544F3AF04A476A654DB81B1CC9D76B06A686,199


This way we have a list of high transactional cards.

## Replacing NULL Payer IDs with another indicator of identity:

So, we can identify an user by their id, or their email and their card. But we want to fill that null values with an identity, in this case, we have lots of other transactions made with same emails and cards. We had de discussion of choosing one identifier over the others these are the caveats:
* payer_id: there are 1.3 M nulls.
* card_id: only available on transactions made with credit card
* email: Lots of nulls

The company told us that that number is the document of identity, so what we are going to do is try to fill the voids with information already in the database. 

First, identify emails related with payer_ids, then do the same for card_ids.

In [ ]:
# How many transactions dont have payer_id?
indi_codes = bd[bd['transaction_payer_id'].isna()].copy()
len(indi_codes)

1324483

In [ ]:
# At first, How many transactions dont have any way of identify the user?
indi_codes.drop(indi_codes[(bd['card_id'].isna()) & (bd['transaction_payer_email'].isna())].index, inplace=True)
len(indi_codes)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


1276559

In [ ]:
len(HighIDsCards.index)

4371

In [ ]:
# How many cards can I extract from the list of users without payer_id?
len(list(indi_codes.groupby('card_id').count().index))

48988

In [ ]:
# How many cards, used by Null Users, are used only by one User?
len(indi_codes.drop_duplicates(subset='card_id', keep=False)['card_id'])

14224

In [ ]:
%%time
# How many of these unique card_ids are related with many users?
len([x for x in (list(indi_codes.drop_duplicates(subset='card_id', keep='first')['card_id'].unique())) if x in list(HighIDsCards.index)])

CPU times: user 47.6 s, sys: 25.3 ms, total: 47.6 s
Wall time: 48.3 s


100

#### Replace Function

In [ ]:
emailList=[x for x in list(indi_codes[indi_codes['transaction_payer_email'].notnull()].groupby('transaction_payer_email').count().index)]
len(emailList)

3016

In [ ]:
ID_DF = bd[bd['transaction_payer_email'].isin(emailList)]
len(ID_DF)

18550

In [ ]:
ID_DF = ID_DF.drop(ID_DF[ID_DF['transaction_payer_id'].isna()].index)
len(ID_DF)

9635

In [ ]:
# emails useed by too many payer_ids
no_list = ID_DF.groupby(['transaction_payer_email','transaction_payer_id']).count().reset_index().groupby('transaction_payer_email').count().sort_values(by = 'transaction_payer_id', ascending=False).head(1)#.value_counts()
no_list = (no_list.reset_index())['transaction_payer_email'].values.tolist()
no_list

['L-BP?2G;K$6I.4=WH.WA]/A.>"71Y?,OC@0Y&Y+UW$-']

In [ ]:
ID_DF.drop(ID_DF[ID_DF['transaction_payer_email'].isin(no_list)].index, inplace=True)

In [ ]:
ID_DF.drop_duplicates(subset=['transaction_payer_email', 'transaction_payer_id'], keep='first', inplace=True)
len(ID_DF.drop_duplicates(subset='transaction_payer_email', keep='first'))

1410

In [ ]:
ID_DF_dic = dict(zip(ID_DF['transaction_payer_email'],ID_DF['transaction_payer_id']))

Then, do the same with card_id:

In [ ]:
CardList=[x for x in list(indi_codes[indi_codes['card_id'].notnull()].groupby('card_id').count().index) if x not in list(HighIDsCards.index)]
len(CardList)
Card_DF = bd[bd['card_id'].isin(CardList)]
len(Card_DF)
Card_DF = Card_DF.drop(Card_DF[Card_DF['transaction_payer_id'].isna()].index)
len(Card_DF)
Card_DF.drop_duplicates(subset='card_id', keep='first', inplace=True)
len(Card_DF.drop_duplicates(subset='card_id', keep='first'))
ID_DF_dic2 = dict(zip(Card_DF['card_id'],Card_DF['transaction_payer_id']))

In [ ]:
# defining a function to use with apply()
# This function uses apply over a 'horizontal' series, that is, over a row values. Each column is converted into an element in a list

def replace_payerID(row):
  if row[8] in ID_DF_dic:
    return ID_DF_dic[row[8]]
  elif row[25] in ID_DF_dic2:
    return ID_DF_dic2[row[25]]
  else:
    return row[6]

#### Testing the function

In [ ]:
testDF = indi_codes.head(1000).copy()
testDF.head()
testDF.index

Int64Index([  170,   242,   276,   677,   689,   714,  1138,  1181,  1271,
             1374,
            ...
            25975, 25978, 25981, 25984, 25995, 26002, 26014, 26021, 26026,
            26036],
           dtype='int64', length=1000)

In [ ]:
testDF['transaction_payer_id'].value_counts()

Series([], Name: transaction_payer_id, dtype: int64)

In [ ]:
%%time

testDF['transaction_payer_id'] = testDF.apply(replace_payerID, axis = 1)

CPU times: user 49.7 ms, sys: 999 µs, total: 50.7 ms
Wall time: 56.5 ms


In [ ]:
testDF['transaction_payer_id'].value_counts()

FC,C)%K%$'-8&..?@``MVLC)LV'<Q39?_"=#4<"$N/B    9
MR9"8&[JV?$==861H20F,;N7*P-FECZD[H3`BH8B'ON    8
L88C3PU7)Q`,*S*#W+X:G49YV,X+#O^`FGH9L@6=#7=    5
J#RS;BYCB`##H*@V4B9[*9N<N!+?AQ_5\7M1.XB4XT;    3
C@UV=9BL$7W/]MK&FN(W48M74I]5'.F^H08`-IN2(HE    3
                                              ..
B)V.X[=#+%@Z_)PTJ:1/BV9'';E+TO67&F.!DG+NV-R    1
P!K*T5U/Z]-Y=+&Z411%^36P=W%N?8A:XJ;2T5F<58-    1
NKZS`3Z@4G3'QCX;R03HDMB-,9G40A9>,OU^DO1ZHK;    1
BVN'(LT[_RWN#W0T]+4$!X+Z%CP"J*"<HLIS$[(G(!R    1
A+O1NI']UT.U7#S-'),F7MSQEQ7K'*.M&E.9GS;.&`N    1
Name: transaction_payer_id, Length: 257, dtype: int64

In [ ]:
testDF = indi_codes.head(100000)

In [ ]:
%%time
testDF['transaction_payer_id'] = testDF.apply(replace_payerID, axis = 1)

CPU times: user 1.37 s, sys: 23.6 ms, total: 1.39 s
Wall time: 1.42 s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
testDF['transaction_payer_id'].value_counts()

C?C;EQ20MOYE%\QU%M4<SFR\>W2&A&?,>\`6:)Q9*Y-    519
IG`6'FM+;RN%)G`)(,+)7]Q_S;B=`*]_(H^`["TL$Z_    376
I^#T]\G]Y8"'DK4S7Y7BVV,'EYSFFL:<&7)$+E&:K7,    345
B((FX7-H=Q\<@*<$54$UQJ4786CR=62]AN*MI*!FX]5    289
H!.8\5]5V[*:26?)3QIO/LM,,5NX//I039C%V!U-X\3    175
                                              ... 
IG.I-)=[+%)0/=P#@QTWW]&JJN"Z#KQ0=/4X,A"><_R      1
O0+^C)T1(.1\3^\RGBM:.K("6+:^%MUU2^,UD7CYZK5      1
MYZ,9(H;-ETO?HN`&STE;'Z=7.*A=?^!3W1&O+K!Y>+      1
K%[KJRL#T%!DI`1L:.C#@S(R@?Q;3L3.%'FJ<$>`$)9      1
G,HG%[ZXBB[9!P31?^SMC;TA*Q3-$ZDQP`&JYTY<#W-      1
Name: transaction_payer_id, Length: 7588, dtype: int64

#### Applying function

In [ ]:
# Now we are working with a copy of the database:
#testDF = bd[1000000:2000000]
testDF = indi_codes[(indi_codes['transaction_payer_email'].isin(ID_DF_dic) | indi_codes['card_id'].isin(ID_DF_dic2))]
#testDF = bd[bd['transaction_payer_id'].isna()]

#testDF = testDF[~testDF['transaction_payer_id'].notnull()]
testDF = testDF[testDF['transaction_payer_id'].isna()]
testDF.index

Int64Index([     170,      276,      677,     1138,     1436,     1993,
                2270,     2698,     2897,     3598,
            ...
            12324743, 12324746, 12324840, 12324904, 12325016, 12325180,
            12325287, 12325577, 12325738, 12325764],
           dtype='int64', length=337065)

In [ ]:
# Just to verify:
#testDF['transaction_payer_id'].isna().sum()
testDF['transaction_payer_id'].value_counts()

Series([], Name: transaction_payer_id, dtype: int64)

In [ ]:
#testDF.isna().sum()

In [ ]:
%%time
testDF['transaction_payer_id'] = testDF.apply(replace_payerID, axis = 1)

CPU times: user 13.5 s, sys: 6.7 s, total: 20.2 s
Wall time: 36.8 s


In [ ]:
#testDF.isna().sum()

In [ ]:
testDF['transaction_payer_id'].value_counts()

I`P.4&14L:>^_]MT[+M>YJ):=JT4@QF5X@B;S/#9"6*    917
LB*%L*'"3'9&U+;H6B+G\_\ATY#,+Z>N4T(/0WV8C,E    905
BB#E%:2ZR05N7+`>.1\O"8`#_4<9'3Y^V!=X=@54=$A    895
J1E,%+]MH@4YIG-K$%TA-BL#`".JRI?AW[ZPJA$G_9(    882
E2:HZP?RC0&V/7^@O*<!*!)Y*_HPO%@H][BD>4O1BI7    738
                                              ... 
JH//?J*Y!6&*N3@'U]/[=L6SM3R?.6KOPFB[HN2."PU      1
M^P4B0"DQV?U2&+(Z$%)&>?9_KE69Z687S?0;BZYI^Q      1
N#:S^5,C:G.\5\+M29(IB-*KJRI7N$>#&5@,/%9VXH(      1
A`M(++.A=Z+$H2YS\LQ?XNR]+%<68TM)SH5JVV3J:6T      1
B[@@"`N]^@#7%B"^/*!SH<:@:YO+?X58A8'BJ"+UQ.@      1
Name: transaction_payer_id, Length: 15559, dtype: int64

In [ ]:
testDF['transaction_payer_id'].isna().sum()

0

In [ ]:
# the copy was not modified:
indi_codes['transaction_payer_id'].value_counts()

C?C;EQ20MOYE%\QU%M4<SFR\>W2&A&?,>\`6:)Q9*Y-    519
IG`6'FM+;RN%)G`)(,+)7]Q_S;B=`*]_(H^`["TL$Z_    376
I^#T]\G]Y8"'DK4S7Y7BVV,'EYSFFL:<&7)$+E&:K7,    345
B((FX7-H=Q\<@*<$54$UQJ4786CR=62]AN*MI*!FX]5    289
H!.8\5]5V[*:26?)3QIO/LM,,5NX//I039C%V!U-X\3    175
                                              ... 
E]L"8#V"29N"4HIG(=[<Y,+N>NF"/!H5U'`F)EJ_!5*      1
F6H"X!Z&&.:72\?]MA;"`0*B@FQ5`=I6S^5]NDR5K'^      1
IQ"B7,&0\P:U7'<V\FW'CE+HUXZ()`X_"0=L276A.5^      1
D#E^7J(N@GNY85\8KTE[B/>VU+6NJ0@#H0Q1,#9^!HX      1
N.LH'BXI3(,M;P-935%,XF;NOB_5W/N8Q97$HOG?1=P      1
Name: transaction_payer_id, Length: 7588, dtype: int64

In [ ]:
%%time
bd['transaction_payer_id'].isna().sum()

CPU times: user 528 ms, sys: 2.86 ms, total: 531 ms
Wall time: 540 ms


1324483

In [ ]:
indi_codes = pd.concat([indi_codes, testDF])

In [ ]:
indi_codes['transaction_payer_id'].value_counts()

J1E,%+]MH@4YIG-K$%TA-BL#`".JRI?AW[ZPJA$G_9(    979
BB#E%:2ZR05N7+`>.1\O"8`#_4<9'3Y^V!=X=@54=$A    974
LB*%L*'"3'9&U+;H6B+G\_\ATY#,+Z>N4T(/0WV8C,E    963
I`P.4&14L:>^_]MT[+M>YJ):=JT4@QF5X@B;S/#9"6*    917
E2:HZP?RC0&V/7^@O*<!*!)Y*_HPO%@H][BD>4O1BI7    795
                                              ... 
J0R6J:]Z/-;,$0CB6RO,6&43?,3[_-W.O)_-W&J+0AK      1
D'JXEBV:MGNY?XEX:'&HB/^9$5#0W`9#HPS-.2\K=GX      1
K\F,BPP'E5.1::S`G25SRNFW*\Q5O9P&1<"MAQZ9_8>      1
A(:W[ODV:X4>="?`>0(E9.!K656Y54I5G:`_$<NCBF'      1
M1%7DRBVWR/3TSKRU7)55&6Z'3):;M((*F\2P(HN&!B      1
Name: transaction_payer_id, Length: 16717, dtype: int64

In [ ]:
indi_codes.shape

(1587087, 34)

In [ ]:
indi_codes.drop_duplicates(subset=['transaction_id'], keep='last', inplace=True)

In [ ]:
indi_codes.shape

(1276559, 34)

In [ ]:
indi_codes['transaction_payer_id'].value_counts()

J1E,%+]MH@4YIG-K$%TA-BL#`".JRI?AW[ZPJA$G_9(    979
BB#E%:2ZR05N7+`>.1\O"8`#_4<9'3Y^V!=X=@54=$A    974
LB*%L*'"3'9&U+;H6B+G\_\ATY#,+Z>N4T(/0WV8C,E    963
I`P.4&14L:>^_]MT[+M>YJ):=JT4@QF5X@B;S/#9"6*    917
E2:HZP?RC0&V/7^@O*<!*!)Y*_HPO%@H][BD>4O1BI7    795
                                              ... 
O?Q^KYW8LYHX-(YZF!%I?ZNG)5))9E6<`*7>P0LEK4V      1
J\26'GUS>X$);.SBWS`"H;MW)+'/=SZJXT&LRJMR=%C      1
E?B5.Y8I:8_*O>L.6!P#(+II>.2;_J'3YM16M526'Q+      1
P@R:JIH*^.KRTC[:R%$&6W=<5#Q5C!Q.XSX%$V92`B;      1
GHKJNAAKKQ-N(JO!*0:+WWP/:6?&5;$9QT)=.]7%GE8      1
Name: transaction_payer_id, Length: 16717, dtype: int64

In [ ]:
indi_codes['transaction_payer_id'].isna().sum()

939494

In [ ]:
bd['transaction_payer_id'].isna().sum()

1324483

In [ ]:
bd['transaction_payer_id'].value_counts()

A0@*-`=[R2O)DAO12FTZB-,+(&H0Z"84YYVQL61.(*V    2821
PBUH,R%3.`\\R56:1\:ES6DY>3G8&IN)7A<<W]WQR2_    2686
IG`6'FM+;RN%)G`)(,+)7]Q_S;B=`*]_(H^`["TL$Z_    2665
GOB7T&=O6;2$/2*ZK:OHUKNG;[S6YG8(CF7T8AF0SD<    2563
C7(MWE@\*K?"\#9X#LJUOU)";`5HL"N+DU.29;'L,D5    2415
                                               ... 
P$(,FP/%Y=&G'I'D[+;BFJ#E^]<XBF3!DUI<]N`WS_6       1
F^_(SKF1W+GE*&F;C6:Y8`9>=;.4P(0R8V:;<--;NWD       1
M0HE%=[Q[A6:<)`JKL$\3RBX.FZV)W2@51I(LP7H1IQ       1
FC=EN"JM##QJ2C5,8-JF+B7ET+^3R2.^1%,#/HK"90=       1
O)W&L`RC98MH\92&A+S]1!?-D"\:E\_E$+H4$P4T>6I       1
Name: transaction_payer_id, Length: 2903776, dtype: int64

In [ ]:
bd = pd.concat([bd, testDF])
bd.shape

(12647756, 34)

In [ ]:
bd.drop_duplicates(subset=['transaction_id'], keep='last', inplace=True)
bd.shape

(12310691, 34)

In [ ]:
bd['transaction_payer_id'].value_counts()

A0@*-`=[R2O)DAO12FTZB-,+(&H0Z"84YYVQL61.(*V    2824
PBUH,R%3.`\\R56:1\:ES6DY>3G8&IN)7A<<W]WQR2_    2686
IG`6'FM+;RN%)G`)(,+)7]Q_S;B=`*]_(H^`["TL$Z_    2665
GOB7T&=O6;2$/2*ZK:OHUKNG;[S6YG8(CF7T8AF0SD<    2563
C7(MWE@\*K?"\#9X#LJUOU)";`5HL"N+DU.29;'L,D5    2415
                                               ... 
IM5EA=&4=GVUV&VYUOB`B%)M'1+4(+<C2N44]\JQ1G0       1
H<Y9)8PZ)#?.ZLCXEI2(46+QU%_6\:HH^NBF;>#@"2K       1
J^3P8I/+J%]\F"2`1(V9MW]SH32S7?<[T6Q+6$`=XM3       1
B6M`:NQ',Y+//97U/5W"B=N=5T,2>J#OMGN8-MOE4A%       1
O)W&L`RC98MH\92&A+S]1!?-D"\:E\_E$+H4$P4T>6I       1
Name: transaction_payer_id, Length: 2903776, dtype: int64

In [ ]:
bd['transaction_payer_id'].isna().sum()

1013955

## Using emails as identifiers when nothing else

In [ ]:
email_list2 = bd[(bd['transaction_payer_id'].isna())&(bd['transaction_payer_email'].notnull())].index

In [ ]:
bd.loc[email_list2,'transaction_payer_id'] = bd.loc[email_list2,'transaction_payer_email']
bd.loc[email_list2,'transaction_payer_id']

689         H%_S##(9YRPWQN?17JN*50"BBFG6_J).%JK[,*.NH9*
1816        F<Y(*QL'8CP]\!C3:--X!B/9HJ/'4..!!;;@+Y=[D3[
4809        C:\>*ITT4)DBE(*`&8481R/J=C*<NTHV#,92G"O-"-;
6857        FQW^J5[.)#$86GG8QYCD?-$C#73J;#)Z7E\><+59@X5
6980        FQW^J5[.)#$86GG8QYCD?-$C#73J;#)Z7E\><+59@X5
                               ...                     
4806388     LD"D5%X\TH^@%L(?L&YQ:MN["]NY(7@5071W[)%;BO-
4806419     LD"D5%X\TH^@%L(?L&YQ:MN["]NY(7@5071W[)%;BO-
4806491     LD"D5%X\TH^@%L(?L&YQ:MN["]NY(7@5071W[)%;BO-
5389302     DB;C:76!\#IN\W8QGSH&B"`<=%`=@;#<^17)??QJJVF
10459018    DB;C:76!\#IN\W8QGSH&B"`<=%`=@;#<^17)??QJJVF
Name: transaction_payer_id, Length: 3706, dtype: object

## Replacing payer_ids with shorter forms

In [ ]:
len(bd['transaction_payer_id'].unique())

2905369

In [ ]:
names = pd.Series(['I,\\N;OT_', 'N:*&/E+-'])

In [ ]:
names = bd['transaction_payer_id'].copy()

In [ ]:
test_string = ''.join((filter(lambda i: i not in bad_chars, test_string)))

In [ ]:
import re


def Clean_chars(Name): 
    #print(type(Name), Name)
    Name = str(Name)
    Name = re.sub('[^a-zA-Z0-9?]','',Name)
    #Name = Name.replace("'",  r"-")
    #Name = Name.replace('"',  r"-")
    #Name = Name.replace("\\", r"-")
    #Name = Name.replace("`",  r"~")
    #Name = Name.replace(" ",  r"_")
    #Name = Name.replace(";",  r".")
    #Name = Name.replace(",",  r".")                           
    #Name = Name.replace(",",'dd')
    #Name = Name.replace(' ','_').lstrip().rstrip()
    return Name


In [ ]:
bd['transaction_payer_id'] = bd['transaction_payer_id'].apply(Clean_chars).str.slice(start=0, stop=10, step=None)
bd['transaction_payer_id'] = bd['transaction_payer_id'].replace("nan",np.NaN)

## Non identifiable users

In [ ]:
# How many transactions doesnt have any indicator of identity as payer_id, email or card_id?
NullUsers =  bd[(bd['transaction_payer_id'].isna()) & (bd['card_id'].isna()) & (bd['transaction_payer_email'].isna())]
len(NullUsers)

47924

In [ ]:
# In which merchants are those Null users having how many transactions?, and How many different merchants?
NullUsers.groupby('merchant_id').count().sort_values(by = 'transaction_id', ascending = False)

,transaction_user_agent,transaction_id,transaction_description,transaction_processing_date_,transaction_processing_hour,transaction_request_language,transaction_payer_id,transaction_payer_document_type,transaction_payer_email,IP,transaction_processing_amount,transaction_card_issuer_name,transaction_card_type,transaction_card_installments,transaction_response_code,site_id,site_channel,site_category,merchant_enabled,merchant_address_city,merchant_created,merchant_classification,ip_location_region_name,ip_location_city,card_id,card_issuer_country,card_class,card_country,paymentmethod_name,paymentmethod_franchise,paymentmethod_type,isic_division_name,isic_section_name
merchant_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CO0000000199,24915,25004,12859,25004,25004,25004,0,0,0,25004,25004,25004,25004,0,25004,25004,25004,25004,25004,25004,25004,25004,0,0,0,0,0,0,25004,0,25004,25004,25004
CO0000001029,2909,3023,1368,3023,3023,3023,0,0,0,3023,3023,3023,3023,0,3023,3023,3023,3023,3023,3023,3023,3023,0,0,0,0,0,0,3023,0,3023,3023,3023
CO0000000384,2890,2929,622,2929,2929,2929,0,0,0,2929,2929,2929,2929,0,2929,2929,2929,2929,2929,2929,2929,2929,0,0,0,0,0,0,2929,0,2929,2929,2929
CO0000000932,2305,2616,2377,2616,2616,2616,0,0,0,2616,2616,2616,2616,0,2616,2616,2616,2616,2616,2616,2616,2616,0,0,0,0,0,0,2616,0,2616,2616,2616
CO0000001652,0,1709,0,1709,1709,1709,0,0,0,1709,1709,1709,1709,0,1709,1709,1709,1709,1709,1709,1709,1709,0,0,0,0,0,0,1709,0,1709,1709,1709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CO0000000907,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CO0000000904,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CO0000000899,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0




While doing the exploratory analisys, we noticed these issues:

*   There is a ***transaction_payer_id*** that have the same ***transaction_payer_email*** +300 times. We need to confirm the accuracy of this sample.

*   There is also a ***transaction_payer_id*** that have purchases representing the 10% of the complete database. We need to confirm this sample in order to ensure is not a test sample.

*   there are ***transaction_payer_id*** that have the same ***transaction_payer_email*** so we gotta ask the entity about this anomaly.






## Filter Columns

As there is no more use in the email, because we have the ID, and no use into the IP, because it is hashed. Now we can drop the `'transaction_payer_email'` and `'IP'` columns

In [ ]:
bd = bd.drop(['transaction_payer_email', 'IP'], axis = 1)
bd.shape

(12310691, 32)

In [ ]:
bd.memory_usage().sum()

1456499529

# Missing values, Nulls and NaNs

Now there were a lot of Nulls. some are coded as in different ways over different columns in the database. What to do with them?

In [ ]:
bd.isna().sum()

transaction_user_agent              2752592
transaction_id                            0
transaction_description              914237
transaction_processing_date_              0
transaction_processing_hour               0
transaction_request_language              0
transaction_payer_id                      0
transaction_payer_document_type      340799
transaction_processing_amount             0
transaction_card_issuer_name          26680
transaction_card_type                     0
transaction_card_installments       7367427
transaction_response_code                 0
site_id                                   0
site_channel                              0
site_category                             0
merchant_id                               0
merchant_enabled                          0
merchant_address_city                     0
merchant_created                          0
merchant_classification                   0
ip_location_region_name             2377935
ip_location_city                

Missingness plots can be created with the missingno library or using a seaborn.heatmap of your data after a pass of the .isnull() method. 

Explore for a monotone missingness pattern

We decided not to do that, because we cannot research in deep the functionality of PlacetoPay Platform in order to asess the reason for the missing values.




Still we see some missing values but we cannot impute or assign or interpolate any value to them, because that will corrupt the database. Instead, as we have some millions of rows, we can use sampling. Making random samples of the data that is not null is useful because we dont know the reason for the null values to be.

Some possible interpretations:
* transaction_user_agent: What is the reason for the platform to not record the OS and device?

* ip_location_region_name, ip_location_city

* transaction_description: We dont know why the description is inexistent.

* transaction_payer_id: the platform registers the card, but there is no name associated

* transaction_payer_document_type, transaction_card_issuer_name: same as above

* transaction_card_installments, card_id , card_issuer_country, card_class, card_country, paymentmethod_franchise: If the payment is made to debit card or other not creditcard, there is no installments

* isic_division_name and isic_section_name: some merchants dont have a classification

## Inputation of values to the Nulls and NaNs

In [ ]:
# DONT RUN
# We didnt input any values to the Null values yet the reason is that a np.Nan consumes lower memory:

bd['transaction_user_agent'].fillna('No Info')
bd['transaction_description'].fillna('No Info')

bd['transaction_payer_id'].fillna('No Info')
bd['transaction_payer_document_type'].fillna('No Info')

bd['ip_location_region_name'].fillna('No Info')
bd['ip_location_city'].fillna('No Info')

# Transactions without information about cards are the transactions where a card different than 'Credit' was used.
bd['transaction_card_issuer_name'].fillna('No Info')
bd['transaction_card_installments'].fillna('No Info')
bd['card_id'].fillna('No Info')
bd['card_class'].fillna('No Info')
bd['card_country'].fillna('No Info')
bd['paymentmethod_franchise'].fillna('No Info')

bd['isic_division_name'].fillna('No Info')
bd['isic_section_name'].fillna('No Info')

## Identify High transactional and high amount payer_ids and card_ids

In [ ]:
%%time
#¿How many transactions with different transaction payer id there are?
HighTransUsers = bd[['transaction_payer_id', 'merchant_id', 'transaction_processing_amount', 'isic_section_name', 'isic_division_name']].groupby(['transaction_payer_id']).count()

CPU times: user 14.7 s, sys: 400 ms, total: 15.1 s
Wall time: 15.1 s


In [ ]:
HighTransUsers.sort_values(by='merchant_id', ascending = False).head(5)


,merchant_id,transaction_processing_amount,isic_section_name,isic_division_name
transaction_payer_id,,,,
IG6FMRNG7Q,3041,3041,3041,3041
A0R2ODAO12,2827,2827,2827,2827
PBUHR3R561,2686,2686,2686,2686
GOB7TO622Z,2563,2563,2563,2563
C7MWEK?9XL,2415,2415,2415,2415


In [ ]:
HighTransUsers['merchant_id'].describe()

count    2.905195e+06
mean     3.898868e+00
std      1.124336e+01
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      4.000000e+00
max      3.041000e+03
Name: merchant_id, dtype: float64

In [ ]:
HighTransUsers['merchant_id'].quantile([0.90,0.95, 0.99, 0.998, 0.999])

0.900      8.0
0.950     12.0
0.990     28.0
0.998     67.0
0.999    100.0
Name: merchant_id, dtype: float64

In [ ]:
LowTransUsers = HighTransUsers[HighTransUsers['merchant_id']<=70]
LowTransUsers['merchant_id'].describe()

count    2.899915e+06
mean     3.620993e+00
std      5.165915e+00
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      4.000000e+00
max      7.000000e+01
Name: merchant_id, dtype: float64

In [ ]:
HighTransUsers = HighTransUsers[HighTransUsers['merchant_id']>70]
HighTransUsers['merchant_id'].describe()

count    5280.000000
mean      156.515341
std       177.680962
min        71.000000
25%        83.000000
50%       107.000000
75%       155.000000
max      3041.000000
Name: merchant_id, dtype: float64

In [ ]:
len(list(HighTransUsers.index))

5280

In [ ]:
HighTransCards = bd[['transaction_payer_id', 'card_id', 'transaction_id']].groupby(['card_id']).count().sort_values(by = 'transaction_id', ascending = False)

In [ ]:
HighTransCards.describe()

,transaction_payer_id,transaction_id
count,1.259478e+06,1.259478e+06
mean,3.143810e+00,3.143810e+00
std,1.013752e+01,1.013752e+01
min,1.000000e+00,1.000000e+00
25%,1.000000e+00,1.000000e+00
50%,1.000000e+00,1.000000e+00
75%,3.000000e+00,3.000000e+00
max,3.041000e+03,3.041000e+03


In [ ]:
HighTransCards['transaction_id'].quantile([0.90,0.95, 0.99, 0.998, 0.999])

0.900      6.0
0.950      9.0
0.990     30.0
0.998     74.0
0.999    112.0
Name: transaction_id, dtype: float64

In [ ]:
HighTransCards[HighTransCards['transaction_id']>80].describe()

,transaction_payer_id,transaction_id
count,2158.000000,2158.000000
mean,159.796571,159.796571
std,142.201672,142.201672
min,81.000000,81.000000
25%,96.000000,96.000000
50%,117.500000,117.500000
75%,166.750000,166.750000
max,3041.000000,3041.000000


In [ ]:
LowTransCards = HighTransCards[HighTransCards['transaction_id']<=80]
HighTransCards = HighTransCards[HighTransCards['transaction_id']>80]

In [ ]:
len(list(HighTransCards.index))

2158

We can do the same for the transaction_ammount:

In [ ]:
HighAmountUsers = bd[['transaction_payer_id', 'card_id', 'transaction_id', 'transaction_processing_amount']].groupby(['transaction_payer_id']).mean().sort_values(by='transaction_processing_amount', ascending = False)
#HighAmountUsers.head(5)

In [ ]:
HighAmountUsers['transaction_processing_amount'].describe()

count    2.905195e+06
mean     6.023710e+05
std      3.171832e+07
min      0.000000e+00
25%      6.749925e+04
50%      1.961000e+05
75%      4.761000e+05
max      3.671692e+10
Name: transaction_processing_amount, dtype: float64

In [ ]:
HighAmountUsers['transaction_processing_amount'].quantile([0.90,0.95, 0.99, 0.998, 0.999])

0.900    1.053640e+06
0.950    1.800000e+06
0.990    5.355489e+06
0.998    1.335401e+07
0.999    2.037192e+07
Name: transaction_processing_amount, dtype: float64

In [ ]:
LowAmountUsers = HighAmountUsers[HighAmountUsers['transaction_processing_amount']<=15000000]
LowAmountUsers['transaction_processing_amount'].describe()

count    2.900393e+06
mean     4.718025e+05
std      9.785557e+05
min      0.000000e+00
25%      6.730000e+04
50%      1.954427e+05
75%      4.731500e+05
max      1.500000e+07
Name: transaction_processing_amount, dtype: float64

In [ ]:
HighAmountUsers = HighAmountUsers[HighAmountUsers['transaction_processing_amount']>15000000]
HighAmountUsers['transaction_processing_amount'].describe()

count    4.802000e+03
mean     7.946533e+07
std      7.758708e+08
min      1.500059e+07
25%      1.776760e+07
50%      2.312250e+07
75%      3.873317e+07
max      3.671692e+10
Name: transaction_processing_amount, dtype: float64

In [ ]:
len(list(HighAmountUsers.index))

4802

In [ ]:
HighAmountCards = bd[['transaction_payer_id', 'card_id', 'transaction_id', 'transaction_processing_amount']].groupby(['card_id']).mean().sort_values(by='transaction_processing_amount', ascending = False)

In [ ]:
HighAmountCards['transaction_processing_amount'].describe()

count    1.259478e+06
mean     6.233064e+05
std      8.716592e+06
min      0.000000e+00
25%      7.150000e+04
50%      2.343999e+05
75%      5.667500e+05
max      5.980311e+09
Name: transaction_processing_amount, dtype: float64

In [ ]:
HighAmountCards['transaction_processing_amount'].quantile([0.90,0.95, 0.99, 0.998, 0.999])

0.900    1.379800e+06
0.950    2.233521e+06
0.990    6.095630e+06
0.998    1.280402e+07
0.999    1.653633e+07
Name: transaction_processing_amount, dtype: float64

In [ ]:
LowAmountCards = HighAmountCards[HighAmountCards['transaction_processing_amount']<=15000000]
HighAmountCards = HighAmountCards[HighAmountCards['transaction_processing_amount']>15000000]

In [ ]:
len(list(HighAmountCards.index))

1647

That way, we have the following lists:

* **HighTransUsers**:  
    Users that have more than 50 transactions
* **HighTransCards**:  
    Cards that have more than 50 transactions
* **HighAmountUsers**:  
   Users that spend more than \$10.000.000
* **HighAmountCards**:  
   Cards that are charged more than \$10.000.000
* **HighIDsCards**:  \
   Cards that have more than 4 users


that way, we can indicate if a user is one of the high frequent and high amount users or it is a regular user. 

And combining this information with **Document type**, to identify companies or aggregator users, (because they use NIT/RUT, Tax ID, etc)


# Save File DB4

In [ ]:
pwd

'/home/jovyan/work/PROJECT/Data'

In [ ]:
#/content/drive/My Drive/Colab Notebooks

In [ ]:
cd '/content/drive/My Drive/DS4A-3'

/content/drive/My Drive/DS4A-3


In [ ]:
ls

 placetopayDB3.ftr*
 placetopayDB3_pickle*
 placetopayDB4.csv*
 placetopayDB4_pickle*
'Reto MinTIC - Inferencia-y-Recomendacion-personalizada.csv'*
'Reto MinTIC - Inferencia-y-Recomendacion Teams 90-67-89.csv'*


In [ ]:
%%time
# To save as Pickle format (in Jupyter):
bd.to_pickle("./placetopayDB4_pickle")

CPU times: user 18.7 s, sys: 11.6 s, total: 30.3 s
Wall time: 1min 6s


In [ ]:
%%time
# https://medium.com/@yugdamor/solved-your-session-crashed-after-using-all-available-ram-google-colab-e31f6a192d52
# To save as Pickle format (in Colab):
filename = "./placetopayDB4_pickle"
outfile = open(filename,'wb')

pickle.dump(bd,outfile)
outfile.close()

In [ ]:
#load the pickle again:
%%time
DATA_PATH = "/content/drive/My Drive/DS4A-3/Place to pay - DS4A - Databases and Notebooks/placetopayDB4_pickle"
infile = open(DATA_PATH,'rb')
bd = pickle.load(infile)
bd.shape

CPU times: user 11.6 s, sys: 5.07 s, total: 16.6 s
Wall time: 58.8 s


# Final shaping of the Database

In [ ]:
bd['transaction_processing_hour'] = pd.to_numeric(bd['transaction_processing_hour'], downcast='integer')
bd['transaction_card_installments'] = pd.to_numeric(bd['transaction_card_installments'], downcast='integer')
bd.transaction_processing_date_ = pd.to_datetime(bd.transaction_processing_date_)
bd.merchant_created = pd.to_datetime(bd.merchant_created)

## Filter Rows

We proceed to drop all the rows that have no particular use because:
* there is too much missing data, 
* there is no possibility of identify the customer profile, (user id, payer_id, card_id, email)





In [ ]:
# Drop rows where there is no possibility of individualize the user:
#bd.drop(NullUsers.index, axis = 0, inplace =True)
bd.shape

(11326973, 32)

In [ ]:
# Dropping all unidentifiable rows (this is optional)
#bd.dropna(subset=['transaction_payer_id'], inplace=True)
len(bd.dropna(subset=['transaction_payer_id']))

11326973

### Eliminating Transactions made by aggregators

In [ ]:
# transactions made with a business entity ID:
len(bd[bd['transaction_payer_document_type'].isin(['NIT', 'TAX', 'CPJ'])])

805477

In [ ]:
bd.drop(bd[bd['transaction_payer_document_type'].isin(['NIT', 'TAX', 'CPJ'])].index, axis = 0, inplace =True)
bd.shape

(10521496, 32)

In [ ]:
# transactions made with the same single card but having many (hundreds) of individual payer_ids
len(bd[bd['card_id'].isin(HighIDsCards.index.values)])

In [ ]:
bd.drop(bd[bd['card_id'].isin(HighIDsCards.index.values)].index, axis = 0, inplace =True)
bd.shape

In [ ]:
len(bd[bd['card_id'].isin(HighTransCards.index.values)])

9801

In [ ]:
bd.drop(bd[bd['card_id'].isin(HighTransCards.index.values)].index, axis = 0, inplace =True)
bd.shape

(9740289, 32)

In [ ]:
len(bd[bd['card_id'].isin(HighAmountCards.index.values)])

1490

In [ ]:
bd.drop(bd[bd['card_id'].isin(HighAmountCards.index.values)].index, axis = 0, inplace =True)
bd.shape

(9738799, 32)

In [ ]:
len(bd[bd['transaction_payer_id'].isin(HighTransUsers.index.values)])

0

In [ ]:
bd.drop(bd[bd['transaction_payer_id'].isin(HighTransUsers.index.values)].index, axis = 0, inplace =True)
bd.shape

(9738799, 32)

In [ ]:
len(bd[bd['transaction_payer_id'].isin(HighAmountUsers.index.values)])

0

In [ ]:
bd.drop(bd[bd['transaction_payer_id'].isin(HighAmountUsers.index.values)].index, axis = 0, inplace =True)
bd.shape

(9738799, 32)

### yet suspicious transactions


There are some more suspicious rows. We dont know hot to process or differentiate and interpretate transactions that have many Null values or transactions that were not succesful:

Transactions with  `bd['paymentmethod_type']=='CASH'` 

Transactions with bd['response_code'] in ['REJECTED','PENDING', 'FAILED']


In [ ]:
bd.info(null_counts=True)   

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9738799 entries, 3287293 to 2807550
Data columns (total 32 columns):
 #   Column                           Non-Null Count    Dtype         
---  ------                           --------------    -----         
 0   transaction_user_agent           7428035 non-null  object        
 1   transaction_id                   9738799 non-null  object        
 2   transaction_description          8900952 non-null  object        
 3   transaction_processing_date_     9738799 non-null  datetime64[ns]
 4   transaction_processing_hour      9738799 non-null  int8          
 5   transaction_request_language     9738799 non-null  category      
 6   transaction_payer_id             9738799 non-null  object        
 7   transaction_payer_document_type  9585079 non-null  category      
 8   transaction_processing_amount    9738799 non-null  float64       
 9   transaction_card_issuer_name     9714706 non-null  category      
 10  transaction_card_type   

In [ ]:
bd.memory_usage().sum()

897004664



The biggest remarks gathered at the end of that analysis were:



The following transactions were decided to be dropped, because there were no confidence on the data, or a plausible explanation for the data being accurate or useful:

* Transactions with the description as 'test' or 'prueba' except where 'prueba saber'.

* transactions where there was impossible to determine the user was an individual



*   From 34 remaining columns in the dataset, we dropped 2 more: `IP`  and `transaction_payer_email` 

This because we discover these columns are not providing useful information for the characterization and profiling of the users and merchants.

*   Transactions with a particular hash were identified as Null. We droped all.

*   We droped also all the transactions with 'test' in the descriptions

*   We are going to discard IP addresses to create a georeferencing of merchant and users. The reason is due we found many transactions can be crossed around the internet via multiple ip's directions. Therefore by using the user_agent and, ip_region_name, ip_city we can cross validate the local ppurchases in Colombia who has 90% of the quota in their business



# Save File CSV


In [ ]:
%%time
bd.to_csv('placetopayDB4.csv', header=True, index=False)

CPU times: user 9min 56s, sys: 25.5 s, total: 10min 22s
Wall time: 12min 18s


In [ ]:
%%time
# This is a list of user documents that were present in the database. This information should not be in the database due to privacy concerns. 
# because somebody can use this Series for de-anonimize the users and combine with external information to infere attributes about similar users
# those are 6800 users
# 

documents.index
documents.to_csv('documents.csv', header=True, index=False)

In [ ]:
ls

 1-Documentos/                    EDA.ipynb
'2-Deliverable Documents Sent'/  'Luis Hernando'/
'3- EDA Notebooks'/               merchants.csv
'Base Limpia .ipynb'              merchants.gsheet
 bdfn.csv                         payers.csv
 bdfnv2.csv                       payers.gsheet
'Borrador - LuisM.ipynb'          placetopayDB3_pickle
 DB4.csv                          placetopayDB4_pickle
 documents.csv                    PlaceToPayEDA.html


In [ ]:
# We didnt save the tables of high transaction and high amount users, we just use them to filter rows.

In [ ]:
%%time
# $ pip install feather-format
import feather

feather.write_dataframe(bd, "./placetopayDB4.ftr")
#bd.to_feather(FilePath) # do the same, alternatively

CPU times: user 16.1 s, sys: 4.02 s, total: 20.1 s
Wall time: 20.8 s


In [ ]:
# To save as Pickle format:
bd.to_pickle("./placetopayDB4_pickle")

In [ ]:
sample = bd.sample(n=1000000)
sample.to_csv('placetopayDB4_sample.csv', header=True, index=False)

In [ ]:
feather.write_dataframe(sample, "./placetopayDB4_sample.ftr")

In [ ]:
# Converting transaction amount to logaritm
sample['transaction_processing_amount'] = np.log(sample['transaction_processing_amount']+1)

In [ ]:
sample.to_csv('placetopayDB4_sampleLOG.csv', header=True, index=False)

In [ ]:
feather.write_dataframe(sample, "./placetopayDB4_sampleLOG.ftr")